# Creating a simple GDML/Geant4 geometry

## Create a registry 

GDML really is a gloabl namespace for names of geometry objects. This global namescape needs to managed to avoid name clashes etc. This is very important when looking at geometry compositing

In [1]:
from pyg4ometry import geant4 as _g4
from pyg4ometry import visualisation as _vi
from pyg4ometry import gdml as _gdml

Welcome to JupyROOT 6.28/04


In [2]:
reg = _g4.Registry()
reg.clear()

## Create a few NIST materials

In [3]:
worldMaterial = _g4.nist_material_2geant4Material("G4_Galactic", reg)
boxMaterial   = _g4.nist_material_2geant4Material("G4_Au", reg)

## Create some solids

In [4]:
worldSolid = _g4.solid.Box("ws", 100, 100, 100, reg, "mm")
boxSolid   = _g4.solid.Box("bs", 10, 10, 10, reg, "mm")

## Create logical and physical volumes

In [5]:
worldLogical = _g4.LogicalVolume(worldSolid, worldMaterial, "worldLogical", reg)
boxLogical   = _g4.LogicalVolume(boxSolid, boxMaterial, "boxlogical", reg)
boxPhysical  = _g4.PhysicalVolume([0, 0, 0], [0, 0, 0], boxLogical, "boxPhysical1", worldLogical, reg)
reg.setWorld(worldLogical.name)

## Visualise

In [6]:
v = _vi.VtkViewerNew()
v.addLogicalVolume(reg.getWorldVolume())
v.buildPipelinesAppend()
v.view(interactive=True)

## Write to GDML file

In [7]:
w = _gdml.Writer()
w.addDetector(reg)
w.write("box.gdml")

# Parameterised geometry

GDML supports variables which are evaluated when the geometry is used. 

In [49]:
from pyg4ometry import geant4 as _g4
from pyg4ometry import visualisation as _vi
from pyg4ometry import gdml as _gdml
reg = _g4.Registry()
reg.clear()

In [50]:
worldSize = _gdml.Constant("boxSize", 100, reg)
boxSize   = _gdml.Constant("worldSize", 10, reg)

In [51]:
# materials 
worldMaterial = _g4.nist_material_2geant4Material("G4_Galactic", reg)
boxMaterial   = _g4.nist_material_2geant4Material("G4_Au", reg)

# solids 
worldSolid = _g4.solid.Box("ws", worldSize, worldSize, worldSize, reg, "mm")
boxSolid = _g4.solid.Box("bs", boxSize, boxSize, boxSize, reg, "mm")

# structure
worldLogical = _g4.LogicalVolume(worldSolid, worldMaterial, "worldLogical", reg)
boxLogical   = _g4.LogicalVolume(boxSolid, boxMaterial, "boxlogical", reg)
boxPhysical  = _g4.PhysicalVolume([0, 0, 0], [0, 0, 0], boxLogical, "boxPhysical1", worldLogical, reg)

# visualise
reg.setWorld(worldLogical.name)
v = _vi.VtkViewerNew()
v.addLogicalVolume(reg.getWorldVolume())
v.buildPipelinesAppend()
v.view(interactive=True)

### Calculations using parameters

In [52]:
boxSize = worldSize/10

In [53]:
boxSize

Constant : var_boxSize_div_10.000000000000000 = expr_var_boxSize_div_10.000000000000000 : (boxSize) / (10.000000000000000)

In [54]:
import numpy as _np
reg = _g4.Registry()
reg.clear()

# materials 
worldMaterial = _g4.nist_material_2geant4Material("G4_Galactic", reg)
boxMaterial   = _g4.nist_material_2geant4Material("G4_Au", reg)

# solids 
worldSolid = _g4.solid.Box("ws", worldSize, worldSize, worldSize, reg, "mm")
boxSolid = _g4.solid.Box("bs", boxSize, boxSize, boxSize, reg, "mm")

ringRadius = _gdml.Constant("ringSize", 50, reg)

# structure
worldLogical = _g4.LogicalVolume(worldSolid, worldMaterial, "worldLogical", reg)
boxLogical   = _g4.LogicalVolume(boxSolid, boxMaterial, "boxlogical", reg)

for i in range(0,20,1) :
    ic = _gdml.Constant("i_"+str(i),i/20,reg)
    phi = 2*_np.pi*ic
    x = ringRadius*_gdml.cos(phi)
    y = ringRadius*_gdml.sin(phi)
    
    boxPhysical  = _g4.PhysicalVolume([0, 0, -phi], [x, y, 0], boxLogical, "boxPhysical_"+str(i), worldLogical, reg)

# visualise
reg.setWorld(worldLogical.name)
v = _vi.VtkViewerNew()
v.addLogicalVolume(reg.getWorldVolume())
v.buildPipelinesAppend()
v.view(interactive=True)

Important to note that Constants are written to the GDML file

In [55]:
w = _gdml.Writer()
w.addDetector(reg)
w.write("boxRing.gdml")

In [56]:
!cat boxRing.gdml

# Loading, understanding and  editing geometry

In [1]:
from pyg4ometry import gdml as _gdml

Welcome to JupyROOT 6.28/04


In [5]:
r = _gdml.Reader("box.gdml")
reg = r.getRegistry()

In [10]:
reg.solidDict

{'ws': Box : ws Expression : ws_pX = expr_ws_pX : 100 Expression : ws_pY = expr_ws_pY : 100 Expression : ws_pZ = expr_ws_pZ : 100,
 'bs': Box : bs Expression : bs_pX = expr_bs_pX : 10 Expression : bs_pY = expr_bs_pY : 10 Expression : bs_pZ = expr_bs_pZ : 10}

In [13]:
reg.materialDict

{'G4_H_1': <Isotope: G4_H_1>,
 'G4_H_2': <Isotope: G4_H_2>,
 'G4_H': <Element: G4_H>,
 'G4_Au': <Element: G4_Au>,
 'G4_Galactic': <Material: G4_Galactic>,
 'Material_G4_Au': <Material: Material_G4_Au>}

In [16]:
worldLogical = reg.getWorldVolume()

# Importing other formats

In [32]:
def displaySolid(solid) :
    reg = _g4.Registry()
    
    # materials 
    worldMaterial = _g4.nist_material_2geant4Material("G4_Galactic", reg)
    material   = _g4.nist_material_2geant4Material("G4_Au", reg)

    # solids 
    worldSolid = _g4.solid.Box("worldSolid", 25, 25, 25, reg, "mm")

    # structure
    worldLogical = _g4.LogicalVolume(worldSolid, worldMaterial, "worldLogical", reg)
    logical   = _g4.LogicalVolume(solid, material, "boxlogical", reg)
    physical  = _g4.PhysicalVolume([0, 0, 0], [0, 0, 0], logical, "boxPhysical1", worldLogical, reg)

    # visualise
    reg.setWorld(worldLogical.name)
    v = _vi.VtkViewerNew()
    v.addLogicalVolume(reg.getWorldVolume())
    v.buildPipelinesAppend()
    v.view(interactive=True)

In [13]:
from pyg4ometry import stl as _stl

In [77]:
stl = _stl.Reader("./utah_teapot.stl")
stlSolid = stl.getSolid()
displaySolid(stlSolid)

In [ ]:
from pyg4ometry import pyoce as _pyoce
from pyg4ometry import convert as _con

cad        = _pyoce.Reader("./bodies.step")
freeShapes = cad.freeShapes()
label = freeShapes.Value(1)
shape = cad.shapeTool.GetShape(label)

reg = _g4.Registry()
tesselatedSolid = _con.oceShape_Geant4_Tessellated("cadSolid",shape,reg)
displaySolid(tesselatedSolid)

# Compositing geometry